# Julia for Power

Topics:
0. Using Packages
1. Import
2. Plot 
3. Write Optimization Problem
4. Plot even more

## Using Packages

In Julia use: using "packagename"

In [1]:
using JuMP # used for mathematical programming
using Cbc # used as the LP solver
using DataFrames #used to create tables
using CSV # used to import from CSV
using Plots # used to create Plots
using StatPlots

## How to Import

All we need are some CSV, Excel or Matlab data-files to get the fun started<br>
Import all data in the subfolder into tables to a dictionary structure


In [2]:
data=Dict()
for dataname in readdir("Power_Data")
 data[split(dataname,'.')[1]]=CSV.read("Power_Data/$dataname", delim=';')
end

## How to Plot
Plot a little of the input data by plotting the installed capacities of wind, pv and ror on a map like graph

In [12]:
plotlyjs()
loc=scatter()
loclist=["off" "grey" "Wind-Offshore"; "on" "blue" "Wind-Onshore"; "pv" "yellow" "PV"; "ror" "red" "Water" ]
for i=1:size(loclist,1)
    loc=scatter!(data["node_data"][:,:Long],data["node_data"][:,:Lat],ms=(log.(data["node_data"][:,Symbol(loclist[i,1])].+1)),alpha=0.8,color=loclist[i,2],label=loclist[i,3])
end
loc

## Finally some Optimization
In this cell we create  function solve_ed, which solves the economic dispatch problem for a given set of input parameters.

In [4]:
function solve_el(data)
    #Define different stets
    tim=1:size(data["h_demand"],1)
    gen=1:size(data["plant_con"],1)
    nod=1:size(data["node_data"],1)
    #Define the economic dispatch (ED) model
    el=Model(solver=CbcSolver())

    # Define decision variables
    @variable(el, 0 <= G[p=gen, tim] <= data["plant_con"][p,:cap]) # power output of generators
    @variable(el, 0 <= W[n=nod, t=tim] <= (data["node_data"][n,:on]+data["node_data"][n,:off])*data["h_wind"][t,Symbol(data["node_data"][n,:dena])]) # wind power injection
    @variable(el, 0 <= PV[n=nod, t=tim] <= data["node_data"][n,:pv]*data["h_pv"][t,Symbol(data["node_data"][n,:dena])]) # wind power injection

    # Define the objective function
    @objective(el, Min, sum(data["h_price"][t,Symbol(data["plant_con"][p,:fuel])] * G[p,t] for p=gen for t=tim))

    # Define the power balance constraint
    @constraint(el, sum(G[p, t=tim] for p=gen)+sum(W[n, t=tim]+PV[n, t=tim] for n=nod).== data["h_demand"][t=tim,:demand])

    # Solve statement
    solve(el)

    # return the optimal value of the objective function and its minimizers
    return getvalue(G), getvalue(W), getvalue(PV), getobjectivevalue(el)
end

solve_el (generic function with 1 method)

In [6]:
(g_opt, w_opt, pv_opt, obj)=solve_el(data);

## More Plotting
Visualize Demand, residual and production of each powerplant

In [10]:
Dem=plot(data["h_demand"][:demand],label="Demand",lw=5)
Dem=plot!(data["h_demand"][:demand]-sum(pv_opt[n,:] for n=1:size(pv_opt[:,1],1))-sum(w_opt[n,:] for n=1:size(w_opt[:,1],1)),label="Residual",lw=5, color="LightBlue")
Dem=groupedbar!(transpose(g_opt[:,:]), bar_position = :stack, lw=0, label=reshape(data["plant_con"][:Plantname], 1, 98), colour=reshape(colormap("Blues", size(g_opt[:,:],1)),1,size(g_opt[:,:],1)))

In [8]:
function indx(symb, a, b)
    return indexin(data["$a"][Symbol(symb)],data["$b"][Symbol(symb)])
end

indx (generic function with 1 method)

## Plotting the Plants

In [11]:
map=scatter(data["node_data"][indx("nodeno","plant_con","node_data"),:Long],data["node_data"][indx("nodeno","plant_con","node_data"),:Lat],ms=g_opt[:,1]./100,group=data["plant_con"][:fuel])
map